# Hyplex code

In [1]:
import types
from ultralytics import YOLO
import io
import os
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.path as mpltPath
from scipy.spatial import Voronoi
import colorsys

In [2]:
def get_color_by_id(point_region_id, total_ids):
    hue = point_region_id / total_ids # Scale the hue by the number of unique IDs, wrapping around the hue circle
    saturation = 0.9; value = 0.9  # Keep saturation and value high for bright colors
    rgb = colorsys.hsv_to_rgb(hue, saturation, value)
    return tuple(int(c * 255) for c in rgb) # Convert to 0-255 scale for RGB colors in PIL

def get_vor_boundaries(boxes, ImgIfDesired = None):
    points= []; vor_verts = {}
    for idx in range(len(boxes)): points.append((boxes[idx][0], boxes[idx][1]))
    vor_verts_list = []
    try:
        vor = Voronoi(points)
    except:
        return vor_verts_list
    for point_region_id, region_id in enumerate(vor.point_region): #this is needed to preserve the order
        if (-1 not in vor.regions[region_id]):
            region_vertices = vor.vertices[vor.regions[region_id]]
            vor_verts[point_region_id] = region_vertices.tolist()

    default_triangle_height = 2; default_triangle_base_length = 4
    for idx, point in enumerate(points):
        if idx in vor_verts:  # Voronoi region exists
            vor_verts_list.append(vor_verts[idx])
        else:  # Create default triangle for missing regions
            bl_vertex = (point[0] - default_triangle_base_length / 2, point[1])
            br_vertex = (point[0] + default_triangle_base_length / 2, point[1])
            top_vertex = (point[0], point[1] + default_triangle_height)  
            vor_verts_list.append([bl_vertex, br_vertex, top_vertex])
    
    if (ImgIfDesired):
        drawV = ImageDraw.Draw(ImgIfDesired)
        r = 2  # radius of the points
        for point_region_id, point in enumerate(points):
            outline_color = get_color_by_id(point_region_id, len(points))
            left_up_point = (point[0] - r, point[1] - r)
            right_down_point = (point[0] + r, point[1] + r)
            if vor_verts.get(point_region_id) and len(vor_verts[point_region_id]) > 0:
                polygon_vertices_tuples = [tuple(vertex) for vertex in vor_verts[point_region_id]]
                drawV.polygon(polygon_vertices_tuples, width=3, outline=outline_color)
            drawV.ellipse([left_up_point, right_down_point], fill=outline_color)

        display(imgV)
    return vor_verts_list

def find_mask_intensities(img_data, image_array, file_name, fraction_complete = 0,shift_x = 0, shift_y = 0, full_width = -1, full_height = -1, include_headers = True, meta_name = "NA", tile_name = "NA", max_bb_area = 99999, macro_model_list = []):
    sto = io.StringIO()
    sth = ''; d = '\t'

    def bstr_h(sth1):
        nonlocal sth
        sth += sth1

    def bstr_m(st1):
        sto.write(st1)

    def bstr_m_start():
        nonlocal sth, sto
        st = sth + '\r' + sto.getvalue()
        sto.close()
        sto = io.StringIO()
        sto.write(st)

    def get_mask(vertices):
        polygon_path = mpltPath.Path(vertices) # Create a path object from the vertices
        inside_polygon = polygon_path.contains_points(class_points)
        mask = inside_polygon.reshape(xx.shape) # Reshape the mask back to the image shape
        return mask

    width =image_array.shape[1]; height = image_array.shape[0]; channels = image_array.shape[2]
    boxes = img_data.boxes.cpu()
    img_box_centers = boxes.xywh 
    img_mask_coords = None if img_data.masks is None else img_data.masks.xy
    img_vor_coords = get_vor_boundaries(img_box_centers)

    # Now we want to see if the mask contains anything in teh macro model

    first = include_headers; masks = {}
    print(f"{fraction_complete:.1%}"," > width =",width,"height =",height,"chs =",channels,"boxes =",len(img_box_centers),"vor =",len(img_vor_coords))
    xx, yy = np.meshgrid(np.arange(width),np.arange(height)) # Create a mesh grid of coordinate values
    x_flat = xx.flatten(); y_flat = yy.flatten()
    class_points = np.vstack((x_flat, y_flat)).T # Create a list of (x, y) points from the flattened grid
    for idx in range(len(img_box_centers)):
        if (idx % 500 == 499): print("Measuring Intensities",idx)
        bbox_xywh = img_box_centers[idx]
        bbox_corners = [[bbox_xywh[0] - bbox_xywh[2], bbox_xywh[1] + bbox_xywh[3]],[bbox_xywh[0] + bbox_xywh[2], bbox_xywh[1] + bbox_xywh[3]] ,[bbox_xywh[0] + bbox_xywh[2], bbox_xywh[1] - bbox_xywh[3]], [bbox_xywh[0] - bbox_xywh[2], bbox_xywh[1] - bbox_xywh[3]]]
        vor_corners = img_vor_coords[idx] if img_vor_coords else None
        polys = { "box": bbox_corners, "poly": img_mask_coords,  **({"vor": vor_corners} if vor_corners else {}) }
        masks = {key: get_mask(value) for key, value in polys.items() if value}

        cx = bbox_xywh[0].item() + shift_x; cy = bbox_xywh[1].item() + shift_y
        if (first): bstr_h('FileName' + d + 'MetaName' + d + 'TileName' + d + 'ObjectID' + d + 'Class'                         + d + 'Confidence'                + d + 'cx'    + d + 'cy'    + d)
        bstr_m(             file_name + d + meta_name  + d +  tile_name + d + str(idx)   + d + str(int(boxes[idx].cls.item())) + d + str(boxes[idx].conf.item()) + d + str(cx) + d + str(cy) + d)  #I just added the +d at the end on 6/18/2024

        for ma in macro_model_list:
            ry = int(ma.dim_x * cx / full_width)
            rx = int(ma.dim_y * cy / full_height)
            mac_cls, mac_conf = ma.instance_mask[rx,ry,0], ma.instance_mask[rx,ry,1]
            #print(cx,full_width, ma.dim_x, rx, mac_cls, mac_conf)
            if (first): bstr_h("Macro Cls " + ma.name[:8] + d + "Macro Conf " + ma.name[:8] + d)
            bstr_m(     str(int(mac_cls))             + d + str(mac_conf)           + d)

        # Look at each mask for each channel
        for c in range(channels):
            cs = str(c)
            for key in masks:
                selected_pixels = image_array[:, :, c][masks[key]]
                area = len(selected_pixels)
                #TODO: Add in Major Minor (maj, min) = major_minor_axis_lengths(?,?) # Probably only for the polygon masks
                if (first and c==0): bstr_h(key + ' AreaP' + d)
                if (c==0): bstr_m(               str(area) + d)

                sum = np.sum(selected_pixels)
                avg = np.average(selected_pixels)
                std = np.std(selected_pixels)
                if (first): bstr_h(key + ' Total Intensity wv' + cs + d + key + ' Avg Intensity wv' + cs + d + key + ' Std Intensity wv' + cs + d)
                bstr_m(                    str(sum)                 + d + str(avg)                       + d + str(std)                       + d)

        if (first): bstr_m_start(); first = False
        bstr_m('\r')
    return sto.getvalue()

def Predict_OnPartsOfImage(model_simpleType, original_image_name, full_image_arr_predict, full_image_arr_measure = None, save_path = None, save_imgs = None, overlap_amount = 0, fill_edges = False, include_headers = True, meta_name = "NA", testMode = False, maxdets = 6666, macro_model_list = []):
    new_w = model_simpleType.dim_x; new_h = model_simpleType.dim_y
    
    def get_piece(t_arr, x, y):
        piece = t_arr[y:min(y + new_h, t_arr.shape[0]), x:min(x + new_w, t_arr.shape[1])] # Calculate the dimensions of the piece
        if fill_edges: # Create a new array filled with zeros (black) of the desired final size
            filled_piece = np.zeros((new_h, new_w), dtype=t_arr.dtype)
            filled_piece[:piece.shape[0], :piece.shape[1]] = piece
            piece = filled_piece
        return piece
            
    t_arr = full_image_arr_predict
    first = include_headers
    st = io.StringIO()
    w, h = t_arr.shape[1], t_arr.shape[0]
    tPath = os.path.join(save_imgs,cPredImgFldr); os.makedirs(tPath, exist_ok=True)
    total_tiles = (h+1)/(new_h - overlap_amount) * (w+1)/(new_w - overlap_amount); tile_counter = 0.0
    for y in range(0, h, new_h - overlap_amount):
        for x in range(0, w, new_w - overlap_amount):
            tile_counter += 1
            piece_pred = get_piece(t_arr, x, y)
            piece_meas = get_piece(full_image_arr_measure, x, y) if (full_image_arr_measure is not None) else piece_pred
            tilename = str(x) + "," + str(y); print("Region:",tilename)
            predictions = model_simpleType.model.predict(piece_pred, show=False, max_det=maxdets, conf=model_simpleType.min_conf, half=True) 
            if save_imgs is not None: 
                img_array=predictions[0].plot(labels=False, boxes=True, masks=True); Image.fromarray(img_array[..., ::-1]).save(os.path.join(tPath, tilename + ".jpg"))
            st.write(find_mask_intensities(predictions[0], piece_meas, original_image_name, tile_counter/total_tiles, x, y, w, h, first, meta_name, tilename, model_simpleType.max_area, macro_model_list))
            if (testMode and not first): break # This will give us exactly two regions
            first = False
        if (testMode and not first): break

    strRet = st.getvalue()
    if (save_path is not None):
        with open(save_path, 'a') as file: file.write(strRet)
        st.close()
    print("Done with File")
    return strRet

cPredImgFldr = "_PredImgs"
def create_multichannel_array(folder_path, down_x = 1, down_y = 1):
    image_arrays = []
    image_names = []
    for root, dirs, files in os.walk(folder_path):
        if (os.path.basename(root) == cPredImgFldr): continue
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg','.bmp','.tif')):
                print("loading ", file)
                file_path = os.path.join(root, file)
                #t_img = Image.open(file_path).convert('L');  # Convert to grayscale if not already # Old Way
                
                t_img = Image.open(file_path); 
                width, height = t_img.size  
                new_width = int(width * down_x); new_height = int(height * down_y)
                t_img = t_img.resize((new_width, new_height), Image.Resampling.BILINEAR) # Resize the image

                t_arr = np.array(t_img) 
                #max_intensity = np.max(t_arr); 
                #img_8bit = (255.0 * img_array / max_intensity).astype('uint8')
                #img = Image.fromarray(img_8bit).convert('RGB'); width, height = img.size
                
                if t_arr.ndim == 2:  # Ensure the image is grayscale
                    image_arrays.append(t_arr)
                    image_names.append(file)
    
    if not image_arrays:
        return None  # Or raise an exception if you prefer

    # Stack the arrays along a new axis to create a multi-channel array
    multi_channel_array = np.stack(image_arrays, axis=-1)
    return multi_channel_array, image_names

def create_instance_mask(image_width, image_height, yolo_results_data, show_test_mask = False):
    """
    Create a mask where each pixel value represents the instance class.

    :param image_shape: A tuple (height, width) representing the image shape.
    :param masks: The masks from the YOLOv8 results.
    :param classes: The classes of each instance.
    :return: A mask array where each pixel value represents the instance class. First channel is class, 2nd is confidence, 3rd is 0
    """
    boxes = yolo_results_data.boxes.cpu()
    masks = yolo_results_data.masks.cpu()
    classes_conf = [(boxes[idx].cls.item(),boxes[idx].conf.item()) for idx in range(len(boxes))]
    instance_mask = np.zeros((image_width, image_height, 3), dtype=float) # dtype=np.int32)

    for idx in range(len(boxes)):
        poly_cors = masks[idx].xy[0]
        path = mpltPath.Path(poly_cors)
        y, x = np.mgrid[:image_width, :image_height]
        points = np.vstack((x.ravel(), y.ravel())).T
        mask = path.contains_points(points)
        mask = mask.reshape((image_width, image_height))
        #instance_mask[mask] = classes_conf[idx] 
        instance_mask[mask, 0] = classes_conf[idx][0]
        instance_mask[mask, 1] = classes_conf[idx][1]
    if (show_test_mask):
        plt.figure()
        plt.imshow(instance_mask, cmap='gray')
        plt.title('Mask')
        plt.show()
    return instance_mask

def find_first_file(m_folder, m_contains):
    for root, dirs, files in os.walk(m_folder):
        for file in files:
            if m_contains in file:
                return os.path.join(root, file)
    return None

def work_on_folder(model_simpleType, macro_model_list, SubFolder, PredContains, down_x = 1, down_y = 1, testMode = False, IncludeHeaders = True, save_path = None, maxdet = 10000, overlap = 0):
    file_pred = find_first_file(SubFolder, PredContains)
    
    if file_pred is None:
        return None

    # Prep the large image for micro predictions
    img = Image.open(file_pred); img_array = np.array(img); # This code can handle 8 and 16-bit TIFFs . . not sure about RGB
    # this line is giving an error, fil_pred is type None for some reason, why is this?
    #   well, file_pred is defined on line 232
    #   bruh, find_first_file returns None for some reason
    #   oh, only if m_contains isn't in m_folder
    #   so, I'm getting an error because the file I'm looking for isn't in that folder
    #       let's check what file and folder I was looking at
    # So the folder I'm looking at has multiple subfolders
    #   so it looks like find_first_file only looks at the set of files in a folder?
    #   shouldn't it be that find_first_file looks in all the subdirectories?
    
    max_intensity = np.max(img_array); img_8bit = (255.0 * img_array / max_intensity).astype('uint8')
    img = Image.fromarray(img_8bit).convert('RGB'); width, height = img.size
    if (down_x < 1 or down_y < 1):
        new_width = int(width * down_x); new_height = int(height * down_y)
        img = img.resize((new_width, new_height), Image.Resampling.BILINEAR) # Resize the image before micro model
        print("  Resized image from", width, height, " to", new_width, new_height)
    pred_arr_m0 = np.array(img) # Convert to array
    print(" Loaded data from image", img.width, "x", img.height," m", img.mode, " Orig Max",max_intensity, " min", np.min(pred_arr_m0), " max", str(np.max(pred_arr_m0)))
    
    # Prep for Macro predictions (if there are any requested)
    if macro_model_list: #checks both not none and has list elements
        resize_dict = {}
        for ma in macro_model_list:
            key = (ma.dim_x, ma.dim_y)
            res_img = resize_dict.get(key)
            if res_img is None: 
                p_img = img.resize(key, Image.Resampling.BILINEAR)
                res_img  = np.array(p_img); resize_dict[key] = res_img
            print("  Running Macro Model", ma.name, " on resized image")
            ma.res = ma.model.predict(res_img)
            ma.instance_mask = create_instance_mask(ma.dim_x, ma.dim_y, ma.res[0])
            img_array=ma.res[0].plot(labels=False, boxes=True, masks=True)
            tPath = os.path.join(SubFolder, cPredImgFldr); os.makedirs(tPath, exist_ok=True)
            Image.fromarray(res_img).save(os.path.join(tPath, "orig1.jpg"))
            Image.fromarray(img_array[..., ::-1]).save(os.path.join(tPath, "macro1.jpg"))
            img.save(os.path.join(tPath, "orig1.jpg"))

    meas_arr_m0, names = create_multichannel_array(SubFolder, down_x, down_y)
    st = Predict_OnPartsOfImage(model_simpleType, file_pred, pred_arr_m0, meas_arr_m0, None, SubFolder, overlap, False, IncludeHeaders, SubFolder, testMode, maxdet, macro_model_list) #st = Predict_OnPartsOfImage(m0.model, file_pred, pred_arr_m0, meas_arr_m0, None, dim_x, dim_y, 0, False, IncludeHeaders, SubFolder, testMode, maxdet, m0.min_conf, m0.max_area) # Old Style
    if (save_path is not None): 
        with open(save_path, 'a') as file: file.write(st)
    print("Done with Files")
    return st, names


In [3]:
#Creed

model_path = r"S:\Phys\FIV925 XSection\Datasets\Creed\01a\YO 553 0328 MAXI\map75=0296662 yolov9c  idx=1 ep=8 btch=16 rnd=4717152\weights\best.pt"
m_folder = r"S:\Phys\FIV925 XSection\Datasets\Creed\20240406"
m_contains = "T0_DAPI"
res_append = "2"
dim_x, dim_y = 553, 553
down_x, down_y = 1, 1
max_area = 1000
min_conf = 0.2
testMode = False

In [4]:
#Dougherty Allen

model_path = r"S:\Phys\FIV925 XSection\Datasets\Brain\01a\YO 384 0326 MAXI\map75=0650739 yolov9c  idx=12 ep=11 btch=8 rnd=6376250\weights\best.pt"
m_folder = r"S:\Phys\FIV925 XSection\Datasets\Brain\Dougherty\Orig" # Needs to be in one more subfolder after that
m_contains = "DAPI"
res_append = "2"
dim_x, dim_y = 384, 384
down_x, down_y = 0.5, 0.5
max_area = 30000
min_conf = 0.5
testMode = False

#Implement intensity threshold to show preview images

In [5]:
# Spinal Cord 2 Scales

m_folder = r"S:\Phys\FIV925 XSection\Datasets\SpinalAtlas\Dodd and Fiederling\Cropped\2\R2" # Needs to be in one more subfolder after that
m_contains = "C3" #This is what to focus the segmentation (micro) model on, usually DAPI
res_append = "4"  #This just changes the name of the save folder
testMode = False
m0_down_x, m0_down_y = 1, 1   # Specifies specific ratio of downsampling before starting

# We need a metadata system like we have for the other models, that way we can load all of the parameters about using the model without having to specify them
#if 'm2' in locals(): m2.model = YOLO(m2.model_path), if hasattr(m1, 'model'): # This checks if a member is present (m1.model for example)

# - - - - Micro Level
m0 = types.SimpleNamespace()
m0.model_path = r"S:\Phys\FIV925 XSection\Datasets\Brain\01c\YO 432 0515 Yeti\map75=0669758 yolov9c .pt idx=4 ep=8 btch=8 rnd=4229985\weights\last.pt" 
m0.dim_x, m0.dim_y = 448, 448  # Model input dimensions (will tile this)
m0.max_area = 30000
m0.min_conf = 0.32

# - - - - - Macro Levels
ms = []; ms.append(types.SimpleNamespace())
ms[0].model_path = r"S:\Phys\FIV925 XSection\Datasets\AllenBA\02b\YO 512 0515 MAXI\map75=0804727 YOLOv8n-seg .yaml idx=17 ep=57 btch=32 rnd=8367787\weights\best.pt"
ms[0].dim_x, ms[0].dim_y = 512, 512  # Model input dimensions (will tile this)
ms[0].max_area = 30000000
ms[0].min_conf = 0.20

In [6]:
# Load models
m0.model = YOLO(m0.model_path)
for m in ms: 
    m.model = YOLO(m.model_path); m.name = os.path.basename(os.path.dirname(os.path.dirname(m.model_path)))

Image.MAX_IMAGE_PIXELS = None #Turn off the limit
first_level_subfolders = next(os.walk(m_folder))[1]  # Get first level of folders only
First = True; stio = io.StringIO(); namedict = {}
for subfolder in first_level_subfolders:
    print(subfolder,"---------------------------------")
    subfolder_path = os.path.join(m_folder, subfolder)
    
    # subfolder_path is SubFolder
    # and m_contains is PredContains
    st, names = work_on_folder(m0, ms, subfolder_path, m_contains, m0_down_x, m0_down_y, testMode, First)
    
    stio.write(st); namedict[subfolder] = names; First = False
    if (testMode): break

# Save out the main data
Prefix = f"Res00{res_append}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
save_path = os.path.join(m_folder, Prefix + ".txt"); strRet = stio.getvalue(); stio.close()
with open(save_path, 'a') as file: file.write(strRet)

# Now save out the name information
save_path = os.path.join(m_folder, Prefix + "_Names.txt")
rows = [f"{subfolder}\t{idx}\t{name}" for subfolder, names in namedict.items() for idx, name in enumerate(names)]
with open(save_path, 'w') as txtfile:
    txtfile.write("Subfolder\tIndex\tName\n"); txtfile.write("\n".join(rows))

print("Done with Folder")

_PredImgs ---------------------------------


AttributeError: 'NoneType' object has no attribute 'read'